# Final Model 
Group: Data Trafficerz

Names: Connor Allison, Nicholas Mata, Joel Fry, Royalpriesthood Olola 

In [797]:
import csv
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import numpy as np
np.random.seed(20)
import random
random.seed(20)

X_acc = []
y_acc = []
with open('access_data.csv') as acc_file:
    aCSV = csv.reader(acc_file, delimiter=',')
    for row in aCSV:
        try:
            y_acc.append(int(row[1]))
            X_acc.append(row[0])
        except:
            continue


X_tech = []
y_tech = []
with open('tech_data.csv') as tech_file:
    tCSV = csv.reader(tech_file, delimiter=',')
    for row in tCSV:
        try:
            y_tech.append(int(row[1]))
            X_tech.append(row[0])
        except:
            continue
        

X_acc_train, X_acc_temp, y_acc_train, y_acc_temp = train_test_split(X_acc, y_acc, test_size=0.3, random_state=20)
X_acc_val, X_acc_test, y_acc_val, y_acc_test = train_test_split(X_acc_temp, y_acc_temp, test_size=0.5, random_state=20)

X_tech_train, X_tech_temp, y_tech_train, y_tech_temp = train_test_split(X_tech, y_tech, test_size=0.3, random_state=20)
X_tech_val, X_tech_test, y_tech_val, y_tech_test = train_test_split(X_tech_temp, y_tech_temp, test_size=0.5, random_state=20)

In [847]:
class LexiconClassifier():
    def __init__(self):
        """
            Initalize the Lexicon classifer by loading lexicons. 
        """
        self.access_words = set()
        with open('AccessibilityWords.csv') as iFile:
            iCSV = csv.reader(iFile, delimiter=',')
            for row in iCSV:
                for i in row:
                    self.access_words.add(i.lower().strip())

        self.tech_words = set()
        with open('TechWords.csv') as tFile:
            tCSV = csv.reader(tFile, delimiter=',')
            for row in tCSV:
                for i in row:
                    self.tech_words.add(i.lower().strip())

    def acc_predict(self, sentence):
        num_access_words = 0
        
        for word in sentence.lower().split():
            if word in self.access_words:
                num_access_words += 1
               
        if num_access_words > 0:
            pred = 1
        else:
            pred = 0
            
        return pred
    
    
    def tech_predict(self, sentence):
        num_tech_words = 0
        
        for word in sentence.lower().split():
            if word in self.tech_words:
                num_tech_words += 1
               
        if num_tech_words > 0:
            pred = 1
        else:
            pred = 0
            
        return pred

    
    def count_access_words(self, sentence):
        num_access_words = 0
        for word in sentence.lower().split():
            if word in self.access_words:
                num_access_words += 1
        return num_access_words

    
    def count_tech_words(self, sentence):
        num_tech_words = 0
        for word in sentence.lower().split():
            if word in self.tech_words:
                num_tech_words += 1
        return num_tech_words


In [801]:
lexicon_classifier = LexiconClassifier()


X_acc_train_lexicon_features = [] 
X_acc_val_lexicon_features = [] 
X_acc_test_lexicon_features = [] 

for i in X_acc_train: 
    iList = []
    iList.append(lexicon_classifier.count_access_words(i))
    X_acc_train_lexicon_features.append(iList)
    
for i in X_acc_val: 
    iList = []
    iList.append(lexicon_classifier.count_access_words(i))
    X_acc_val_lexicon_features.append(iList)
    
for i in X_acc_test: 
    iList = []
    iList.append(lexicon_classifier.count_access_words(i))
    X_acc_test_lexicon_features.append(iList)

In [803]:
X_tech_train_lexicon_features = [] 
X_tech_val_lexicon_features = [] 
X_tech_test_lexicon_features = [] 

for i in X_tech_train: 
    iList = []
    iList.append(lexicon_classifier.count_tech_words(i))
    X_tech_train_lexicon_features.append(iList)
    
for i in X_tech_val: 
    iList = []
    iList.append(lexicon_classifier.count_tech_words(i))
    X_tech_val_lexicon_features.append(iList)
    
for i in X_tech_test: 
    iList = []
    iList.append(lexicon_classifier.count_tech_words(i))
    X_tech_test_lexicon_features.append(iList)

In [807]:
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer

vec2 = TfidfVectorizer(ngram_range = (1,1))
vec = CountVectorizer(ngram_range = (1,2), min_df = 3, max_df = 0.95)
X_acc_train2 = vec.fit_transform(X_acc_train)
X_acc_val2 = vec.transform(X_acc_val)
X_acc_test2 = vec.transform(X_acc_test)

X_acc_train_lexicon_features = np.array(X_acc_train_lexicon_features)
X_acc_val_lexicon_features = np.array(X_acc_val_lexicon_features)
X_acc_test_lexicon_features = np.array(X_acc_test_lexicon_features)

X_acc_train_w_lex = sp.hstack([X_acc_train2, X_acc_train_lexicon_features])
X_acc_val_w_lex = sp.hstack([X_acc_val2, X_acc_val_lexicon_features])
X_acc_test_w_lex = sp.hstack([X_acc_test2, X_acc_test_lexicon_features])


X_tech_train2 = vec2.fit_transform(X_tech_train)
X_tech_val2 = vec2.transform(X_tech_val)
X_tech_test2 = vec2.transform(X_tech_test)

X_tech_train_lexicon_features = np.array(X_tech_train_lexicon_features)
X_tech_val_lexicon_features = np.array(X_tech_val_lexicon_features)
X_tech_test_lexicon_features = np.array(X_tech_test_lexicon_features)

X_tech_train_w_lex = sp.hstack([X_tech_train2, X_tech_train_lexicon_features])
X_tech_val_w_lex = sp.hstack([X_tech_val2, X_tech_val_lexicon_features])
X_tech_test_w_lex = sp.hstack([X_tech_test2, X_tech_test_lexicon_features])

Accessibility Model Validation

In [852]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


svc_acc = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_acc = {
    'C': [ 0.001, 0.01, 0.1, 1, 10],
    }

skf = StratifiedKFold(n_splits=5)
clf_acc = GridSearchCV(svc_acc, params_acc, scoring='f1_macro', cv=skf)
clf_acc.fit(X_acc_train2, y_acc_train)

print(f"Best params for Accessibility: {clf_acc.best_params_}")

acc_val_preds = clf_acc.predict(X_acc_val2)

acc_mac_val_precision = precision_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_recall = recall_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_f1 = f1_score(y_acc_val, acc_val_preds, average='macro')
acc_mic_val_precision = precision_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_recall = recall_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_f1 = f1_score(y_acc_val, acc_val_preds, average='micro')
acc_val_mis = 1 - accuracy_score(y_acc_val, acc_val_preds)
print("Acc Val Macro Precision: {:.4f}".format(acc_mac_val_precision))
print("Acc Val Macro Recall: {:.4f}".format(acc_mac_val_recall))
print("Acc Val Macro F1: {:.4f}".format(acc_mac_val_f1))
print("Acc Val Micro Precision: {:.4f}".format(acc_mic_val_precision))
print("Acc Val Micro Recall: {:.4f}".format(acc_mic_val_recall))
print("Acc Val Micro F1: {:.4f}".format(acc_mic_val_f1))
print("Acc Val Misclassification Rate: {:.4f}".format(acc_val_mis))

Best params for Accessibility: {'C': 0.1}
Acc Val Macro Precision: 0.8015
Acc Val Macro Recall: 0.7759
Acc Val Macro F1: 0.7855
Acc Val Micro Precision: 0.8133
Acc Val Micro Recall: 0.8133
Acc Val Micro F1: 0.8133
Acc Val Misclassification Rate: 0.1867


In [811]:
svc_acc = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_acc = {
    'C': [ 0.001, 0.01, 0.1, 1, 10],
    }

skf = StratifiedKFold(n_splits=5)
clf_acc = GridSearchCV(svc_acc, params_acc, scoring='f1_macro', cv=skf)
clf_acc.fit(X_acc_train_lexicon_features, y_acc_train)

print(f"Best params for Accessibility: {clf_acc.best_params_}")

acc_val_preds = clf_acc.predict(X_acc_val_lexicon_features)

acc_mac_val_precision = precision_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_recall = recall_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_f1 = f1_score(y_acc_val, acc_val_preds, average='macro')
acc_mic_val_precision = precision_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_recall = recall_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_f1 = f1_score(y_acc_val, acc_val_preds, average='micro')
acc_val_mis = 1 - accuracy_score(y_acc_val, acc_val_preds)
print("Acc Val Macro Precision: {:.4f}".format(acc_mac_val_precision))
print("Acc Val Macro Recall: {:.4f}".format(acc_mac_val_recall))
print("Acc Val Macro F1: {:.4f}".format(acc_mac_val_f1))
print("Acc Val Micro Precision: {:.4f}".format(acc_mic_val_precision))
print("Acc Val Micro Recall: {:.4f}".format(acc_mic_val_recall))
print("Acc Val Micro F1: {:.4f}".format(acc_mic_val_f1))
print("Acc Val Misclassification Rate: {:.4f}".format(acc_val_mis))

Best params for Accessibility: {'C': 0.01}
Acc Val Macro Precision: 0.7649
Acc Val Macro Recall: 0.6572
Acc Val Macro F1: 0.6652
Acc Val Micro Precision: 0.7467
Acc Val Micro Recall: 0.7467
Acc Val Micro F1: 0.7467
Acc Val Misclassification Rate: 0.2533


In [854]:
svc_acc = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_acc = {
    'C': [ 0.001, 0.01, 0.1, 1, 10],
    }

skf = StratifiedKFold(n_splits=5)
clf_acc = GridSearchCV(svc_acc, params_acc, scoring='f1_macro', cv=skf, refit=True)
clf_acc.fit(X_acc_train_w_lex, y_acc_train)

print(f"Best params for Accessibility: {clf_acc.best_params_}")

acc_val_preds = clf_acc.predict(X_acc_val_w_lex)

acc_mac_val_precision = precision_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_recall = recall_score(y_acc_val, acc_val_preds, average='macro')
acc_mac_val_f1 = f1_score(y_acc_val, acc_val_preds, average='macro')
acc_mic_val_precision = precision_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_recall = recall_score(y_acc_val, acc_val_preds, average='micro')
acc_mic_val_f1 = f1_score(y_acc_val, acc_val_preds, average='micro')
acc_val_mis = 1 - accuracy_score(y_acc_val, acc_val_preds)
print("Acc Val Macro Precision: {:.4f}".format(acc_mac_val_precision))
print("Acc Val Macro Recall: {:.4f}".format(acc_mac_val_recall))
print("Acc Val Macro F1: {:.4f}".format(acc_mac_val_f1))
print("Acc Val Micro Precision: {:.4f}".format(acc_mic_val_precision))
print("Acc Val Micro Recall: {:.4f}".format(acc_mic_val_recall))
print("Acc Val Micro F1: {:.4f}".format(acc_mic_val_f1))
print("Acc Val Misclassification Rate: {:.4f}".format(acc_val_mis))

Best params for Accessibility: {'C': 0.1}
Acc Val Macro Precision: 0.8053
Acc Val Macro Recall: 0.7900
Acc Val Macro F1: 0.7965
Acc Val Micro Precision: 0.8200
Acc Val Micro Recall: 0.8200
Acc Val Micro F1: 0.8200
Acc Val Misclassification Rate: 0.1800


Technology Model Validation

In [815]:
svc_tech = LinearSVC(max_iter = 12000, dual = False, tol=1e-4)
params_tech = {'C': [0.01, 0.1, 0.5, 1, 2, 10],
              'penalty': ['l1', 'l2'],}

skf = StratifiedKFold(n_splits=5)
clf_tech = GridSearchCV(svc_tech, params_tech, scoring='f1_macro', cv=skf)
clf_tech.fit(X_tech_train_lexicon_features, y_tech_train)

print(f"Best params for Technology: {clf_tech.best_params_}")

tech_val_preds = clf_tech.predict(X_tech_val_lexicon_features)

tech_mac_val_precision = precision_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_recall = recall_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_f1 = f1_score(y_tech_val, tech_val_preds, average='macro')
tech_mic_val_precision = precision_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_recall = recall_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_f1 = f1_score(y_tech_val, tech_val_preds, average='micro')
tech_val_mis = 1 - accuracy_score(y_tech_val, tech_val_preds)
print("Tech Val Macro Precision: {:.4f}".format(tech_mac_val_precision))
print("Tech Val Macro Recall: {:.4f}".format(tech_mac_val_recall))
print("Tech Val Macro F1: {:.4f}".format(tech_mac_val_f1))
print("Tech Val Micro Precision: {:.4f}".format(tech_mic_val_precision))
print("Tech Val Micro Recall: {:.4f}".format(tech_mic_val_recall))
print("Tech Val Micro F1: {:.4f}".format(tech_mic_val_f1))
print("Tech Val Misclassification Rate: {:.4f}".format(tech_val_mis))

Best params for Technology: {'C': 0.01, 'penalty': 'l1'}
Tech Val Macro Precision: 0.8013
Tech Val Macro Recall: 0.8030
Tech Val Macro F1: 0.7999
Tech Val Micro Precision: 0.8000
Tech Val Micro Recall: 0.8000
Tech Val Micro F1: 0.8000
Tech Val Misclassification Rate: 0.2000


In [856]:
svc_tech = LinearSVC(max_iter = 12000, dual = False, tol=1e-4)
params_tech = {'C': [0.01, 0.1, 0.5, 1, 2, 10],
              'penalty': ['l1', 'l2'],}

skf = StratifiedKFold(n_splits=5)
clf_tech = GridSearchCV(svc_tech, params_tech, scoring='f1_macro', cv=skf, refit=True)
clf_tech.fit(X_tech_train2, y_tech_train)

print(f"Best params for Technology: {clf_tech.best_params_}")

tech_val_preds = clf_tech.predict(X_tech_val2)

tech_mac_val_precision = precision_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_recall = recall_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_f1 = f1_score(y_tech_val, tech_val_preds, average='macro')
tech_mic_val_precision = precision_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_recall = recall_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_f1 = f1_score(y_tech_val, tech_val_preds, average='micro')
tech_val_mis = 1 - accuracy_score(y_tech_val, tech_val_preds)
print("Tech Val Macro Precision: {:.4f}".format(tech_mac_val_precision))
print("Tech Val Macro Recall: {:.4f}".format(tech_mac_val_recall))
print("Tech Val Macro F1: {:.4f}".format(tech_mac_val_f1))
print("Tech Val Micro Precision: {:.4f}".format(tech_mic_val_precision))
print("Tech Val Micro Recall: {:.4f}".format(tech_mic_val_recall))
print("Tech Val Micro F1: {:.4f}".format(tech_mic_val_f1))
print("Tech Val Misclassification Rate: {:.4f}".format(tech_val_mis))

Best params for Technology: {'C': 2, 'penalty': 'l1'}
Tech Val Macro Precision: 0.8347
Tech Val Macro Recall: 0.8331
Tech Val Macro F1: 0.8266
Tech Val Micro Precision: 0.8267
Tech Val Micro Recall: 0.8267
Tech Val Micro F1: 0.8267
Tech Val Misclassification Rate: 0.1733


In [874]:
svc_tech = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_tech = {'C': [0.01, 0.1, 0.5, 1, 2, 10],
              'penalty': ['l1', 'l2'],}

clf_tech = GridSearchCV(svc_tech, params_tech, scoring='f1_macro', cv=5)
clf_tech.fit(X_tech_train_w_lex, y_tech_train)

print(f"Best params for Technology: {clf_tech.best_params_}")

tech_val_preds = clf_tech.predict(X_tech_val_w_lex)

tech_mac_val_precision = precision_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_recall = recall_score(y_tech_val, tech_val_preds, average='macro')
tech_mac_val_f1 = f1_score(y_tech_val, tech_val_preds, average='macro')
tech_mic_val_precision = precision_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_recall = recall_score(y_tech_val, tech_val_preds, average='micro')
tech_mic_val_f1 = f1_score(y_tech_val, tech_val_preds, average='micro')
tech_val_mis = 1 - accuracy_score(y_tech_val, tech_val_preds)
print("Tech Val Macro Precision: {:.4f}".format(tech_mac_val_precision))
print("Tech Val Macro Recall: {:.4f}".format(tech_mac_val_recall))
print("Tech Val Macro F1: {:.4f}".format(tech_mac_val_f1))
print("Tech Val Micro Precision: {:.4f}".format(tech_mic_val_precision))
print("Tech Val Micro Recall: {:.4f}".format(tech_mic_val_recall))
print("Tech Val Micro F1: {:.4f}".format(tech_mic_val_f1))
print("Tech Val Misclassification Rate: {:.4f}".format(tech_val_mis))

Best params for Technology: {'C': 1, 'penalty': 'l1'}
Tech Val Macro Precision: 0.8160
Tech Val Macro Recall: 0.8135
Tech Val Macro F1: 0.8066
Tech Val Micro Precision: 0.8067
Tech Val Micro Recall: 0.8067
Tech Val Micro F1: 0.8067
Tech Val Misclassification Rate: 0.1933


Accessibility Model Test

In [866]:
svc_acc = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_acc = {'C': [0.001, 0.01, 0.1, 1, 10]}

skf = StratifiedKFold(n_splits=5)
clf_acc = GridSearchCV(svc_acc, params_acc, scoring='f1_macro', cv=skf)
clf_acc.fit(X_acc_train_w_lex, y_acc_train)

print(f"Best params for Accessibility: {clf_acc.best_params_}")

acc_test_preds = clf_acc.predict(X_acc_test_w_lex)

acc_mac_test_precision = precision_score(y_acc_test, acc_test_preds, average='macro')
acc_mac_test_recall = recall_score(y_acc_test, acc_test_preds, average='macro')
acc_mac_test_f1 = f1_score(y_acc_test, acc_test_preds, average='macro')
acc_mic_test_precision = precision_score(y_acc_test, acc_test_preds, average='micro')
acc_mic_test_recall = recall_score(y_acc_test, acc_test_preds, average='micro')
acc_mic_test_f1 = f1_score(y_acc_test, acc_test_preds, average='micro')
acc_test_mis = 1 - accuracy_score(y_acc_test, acc_test_preds)
print("Acc Test Macro Precision: {:.4f}".format(acc_mac_test_precision))
print("Acc Test Macro Recall: {:.4f}".format(acc_mac_test_recall))
print("Acc Test Macro F1: {:.4f}".format(acc_mac_test_f1))
print("Acc Test Micro Precision: {:.4f}".format(acc_mic_test_precision))
print("Acc Test Micro Recall: {:.4f}".format(acc_mic_test_recall))
print("Acc Test Micro F1: {:.4f}".format(acc_mic_test_f1))
print("Acc Test Misclassification Rate: {:.4f}".format(acc_test_mis))

Best params for Accessibility: {'C': 0.1}
Acc Test Macro Precision: 0.8191
Acc Test Macro Recall: 0.7952
Acc Test Macro F1: 0.8055
Acc Test Micro Precision: 0.8467
Acc Test Micro Recall: 0.8467
Acc Test Micro F1: 0.8467
Acc Test Misclassification Rate: 0.1533


In [825]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_acc_test, acc_test_preds)

class_misclassification = {}
for i in range(len(cm)):
    class_total = np.sum(cm[i])
    class_correct = cm[i, i]
    class_misclassification_rate = 1 - (class_correct / class_total)
    class_misclassification[i] = class_misclassification_rate

print("Misclassification Rates per Acc Class:")
for cls, rate in class_misclassification.items():
    print(f"Acc Class {cls}: {rate:.4f}")

Misclassification Rates per Acc Class:
Acc Class 0: 0.0841
Acc Class 1: 0.3256


Technology Model Test

In [878]:
svc_tech = LinearSVC(max_iter = 10000, dual = False, tol=1e-4)
params_tech = {'C': [0.01, 0.1, 0.5, 1, 2, 10],
              'penalty': ['l1', 'l2'],}

skf = StratifiedKFold(n_splits=5)
clf_tech = GridSearchCV(svc_tech, params_tech, scoring='f1_macro', cv=skf)
clf_tech.fit(X_tech_train_w_lex, y_tech_train)

print(f"Best params for Technology: {clf_tech.best_params_}")

tech_test_preds = clf_tech.predict(X_tech_test_w_lex)

tech_mac_test_precision = precision_score(y_tech_test, tech_test_preds, average='macro')
tech_mac_test_recall = recall_score(y_tech_test, tech_test_preds, average='macro')
tech_mac_test_f1 = f1_score(y_tech_test, tech_test_preds, average='macro')
tech_mic_test_precision = precision_score(y_tech_test, tech_test_preds, average='micro')
tech_mic_test_recall = recall_score(y_tech_test, tech_test_preds, average='micro')
tech_mic_test_f1 = f1_score(y_tech_test, tech_test_preds, average='micro')
tech_test_mis = 1 - accuracy_score(y_tech_test, tech_test_preds)
print("Tech Test Macro Precision: {:.4f}".format(tech_mac_test_precision))
print("Tech Test Macro Recall: {:.4f}".format(tech_mac_test_recall))
print("Tech Test Macro F1: {:.4f}".format(tech_mac_test_f1))
print("Tech Test Micro Precision: {:.4f}".format(tech_mic_test_precision))
print("Tech Test Micro Recall: {:.4f}".format(tech_mic_test_recall))
print("Tech Test Micro F1: {:.4f}".format(tech_mic_test_f1))
print("Tech Test Misclassification Rate: {:.4f}".format(tech_test_mis))

Best params for Technology: {'C': 1, 'penalty': 'l1'}
Tech Test Macro Precision: 0.8071
Tech Test Macro Recall: 0.8167
Tech Test Macro F1: 0.8108
Tech Test Micro Precision: 0.8200
Tech Test Micro Recall: 0.8200
Tech Test Micro F1: 0.8200
Tech Test Misclassification Rate: 0.1800


In [835]:
cm = confusion_matrix(y_tech_test, tech_test_preds)

class_misclassification = {}
for i in range(len(cm)):
    class_total = np.sum(cm[i])
    class_correct = cm[i, i]
    class_misclassification_rate = 1 - (class_correct / class_total)
    class_misclassification[i] = class_misclassification_rate

print("Misclassification Rates per Tech Class:")
for cls, rate in class_misclassification.items():
    print(f"Tech Class {cls}: {rate:.4f}")

Misclassification Rates per Tech Class:
Tech Class 0: 0.1702
Tech Class 1: 0.1964
